This is the cretin main file where all the functionality is available. To use this file succesfully you must have added your paths to the paths.py file and you must have cretin installed which is coupled to via the demo.sh.
I will the showcase the full functionality in this file  

In [1]:
from importlib import reload
import generator_object, to_generator_string, search, paths, write_run_plot, serial_sim_tools, import_plt_file #these python classes should be in the same folder as cretin_main
for obj in [generator_object, to_generator_string, search, paths, write_run_plot, serial_sim_tools, import_plt_file]:
    reload(obj)

import numpy as np

In [2]:
# These are all the functions inside the generator object class
gen_object = [i for i in dir(generator_object.User_input()) if i[0:2] != '__']
#print(gen_object)

# all the function inside the to_generator file 
gen_to_file = [i for i in dir(to_generator_string.Text_generator(generator_object.User_input())) if i[0:2] != '__']
#print(gen_to_file)

In [3]:
# This allows us to find frequntly used terms in the generator files, 
# To be specific the lines of text in the generator files get sliced on " " and then selected by the input, f.e 'alias'
# Then a histogram gets made of the most common results
# similarly for most used words in a chapter (ie "Radiation", "Geography" etc)

search_for_word = 'laser'
search_inside_chapter = 'Controls'
df = search.Generators().word(input = search_for_word, word_slicing = (0,10), print_path = False)
#print(df.to_string())
#print(search.Generators().chapter(input = search_inside_chapter))


In [4]:
# reproducing detecterz.gen
gen1 = generator_object.User_input()

k1, k2 = 1, 21
l1, l2 = 1, 11
n2 = (k2*l2)/2 
NI, NE, TE = 10**20, 10**20, 10**3
R0, R1, Z0, Z1  = 0. , 1., -5, 5
RD, ZD = R1 + .1, Z1 +.1
MU1 = 10**-3
MU2 = -1 + MU1
PHI1, PHI2, PHI3 = 1.57, .785, .524

gen1.materials_region(nodes = [k1, k2, l1, l2], elec_temp = 1.1)
gen1.materials_region_background(k1, k2, l1, l2, TE)
gen1.geometry(type = 'rz')
gen1.geometry_quad(node_1 = [k1, k2], node_2= [l1, l2], x_cors = [R0, R1], y_cors = [Z0, Z1], ratios = [1,1])
gen1.radiation_ebins(n_boundaries = 50, start = 100, end = 1000)
gen1.radiation_angles(n_rays = 3, n_angles = 15)
gen1.controls(t_start = 0, t_end = 0)
gen1.popular_switches(continuum_transfer='do steady-state continuum transfer', include_degeneracy='include electron degeneracy')

# this automatically creates the 'trial' folder, writes to the 'trial.gen' file, runs cretin, extracts the data and plots it into the trial/images folder 
write_run_plot.all(name = 'trial1', object = gen1, longprint = False, plot_duplicates = False)


writing to /home/brewster/Desktop/cretin.v2_19_test/bowie/trial1/trial1.gen
running cretin with trial1
plotting trial1 to /home/brewster/Desktop/cretin.v2_19_test/bowie/trial1/images


In [5]:
# Here we reproduce lasfoam.gen from the test folder

gen2 = generator_object.User_input()

N0, N1 = 1, 21 #number of nodes is 20
DN, DR = 10, .1
E1, E2 = 4.68, 4.69
TE, rho = 2.0, 0.003
HPEV = 4.134*10**(-15) #planck in (eV sec)
MULT = HPEV/(E2-E1)

gen2.materials_atom(index = 1, quantum_n_max = 3, element= "h")
gen2.materials_atom(index = 2, quantum_n_max = 3, element= "c")
gen2.materials_region(nodes = [N0, N1], elec_temp = TE)
gen2.materials_region_rho(rho)
gen2.materials_region_element(index = 1, initial_ion_population = 1)
gen2.materials_region_element(index = 2, initial_ion_population = 1)

gen2.geometry(type = 'slab')
gen2.geometry_nodes(coordinate= "r", scaling_type  = "lin", nodes = [N0, N1], nodes_range=[0,DR])

gen2.radiation_ebins(n_boundaries = 61, start = 0.1, end = 10**(5))
gen2.radiation_angles(n_rays = 3)


gen2.popular_switches(timestep_between_snapshot = 5)

gen2.controls(t_start=0., t_end=3*10**-9, restart = True)

# this automatically creates the 'trial' folder, writes to the 'trial.gen' file, runs cretin, extracts the data and plots it into the trial/images folder 
write_run_plot.all(name = 'trial2', object = gen2, longprint = False, plot_duplicates = False)


writing to /home/brewster/Desktop/cretin.v2_19_test/bowie/trial2/trial2.gen
running cretin with trial2
plotting trial2 to /home/brewster/Desktop/cretin.v2_19_test/bowie/trial2/images


In [6]:
# by turning lasfoam into a function we can introduce a variable and do simulations in series using the serial_sim_tools
def lasfoam(variable):
    gen2 = generator_object.User_input()

    N0, N1 = 1, 21 #number of nodes is 20
    DN, DR = 10, .1
    E1, E2 = 4.68, 4.69
    TE, rho = 2.0, 0.003
    HPEV = 4.134*10**(-15) #planck in (eV sec)
    MULT = HPEV/(E2-E1)

    gen2.materials_atom(index = 1, quantum_n_max = 3, element= "h")
    gen2.materials_atom(index = 2, quantum_n_max = 3, element= "c")
    gen2.materials_region(nodes = [N0, N1], elec_temp = TE)
    gen2.materials_region_rho(rho)
    gen2.materials_region_element(index = 1, initial_ion_population = 1)
    gen2.materials_region_element(index = 2, initial_ion_population = 1)

    gen2.geometry(type = 'slab')
    gen2.geometry_nodes(coordinate= "r", scaling_type  = "lin", nodes = [N0, N1], nodes_range=[0,DR])

    gen2.radiation_ebins(n_boundaries = 61, start = 0.1, end = 10**(5))
    gen2.radiation_angles(n_rays = 3)

    gen2.source_laser(laser_wavelength = 4, option_1= 'rate', option_2= 'history', values = [1,variable], nodes= [N0, N1])
    #gen2.source_jnu(E_range=[E1,E2], option_1= 'rate', option_2= 'history', nodes= [N0, N1])
    #gen2.source_jbndry(index = 1, E_range=[E1,E2], option_1= 'rate', option_2= 'history', nodes = [1,1], values=[1,2])

    gen2.popular_switches(timestep_between_snapshot = 5)

    gen2.controls(t_start=0., t_end=3*10**-9, restart = True)
    return gen2

# here i'm showcasing the 'serial_sim_tools.check_dependancy' which checks if the data produced it at all dependent on the variables changing in the for loop
name = 'serial'
i0, iend = 1,5
for i in range(i0,iend):
    variable = i
    # adding our variable to the simulation
    gen2 = lasfoam(variable)
    write_run_plot.all(name = name+str(i), object = gen2, longprint = False, plot_duplicates = False)
    if i > i0:
        org_trial, curr_trial = f'{name}{str(i0)}',f'{name}{str(i)}'
        serial_sim_tools.compare_run(org_trial, curr_trial)


writing to /home/brewster/Desktop/cretin.v2_19_test/bowie/serial1/serial1.gen
running cretin with serial1
plotting serial1 to /home/brewster/Desktop/cretin.v2_19_test/bowie/serial1/images

writing to /home/brewster/Desktop/cretin.v2_19_test/bowie/serial2/serial2.gen
running cretin with serial2
plotting serial2 to /home/brewster/Desktop/cretin.v2_19_test/bowie/serial2/images
comparison of serial1 and serial2, number of identical arrays: 95 number of changed arrays: 0 number of comparision errors: 1

writing to /home/brewster/Desktop/cretin.v2_19_test/bowie/serial3/serial3.gen
running cretin with serial3
plotting serial3 to /home/brewster/Desktop/cretin.v2_19_test/bowie/serial3/images
comparison of serial1 and serial3, number of identical arrays: 95 number of changed arrays: 0 number of comparision errors: 1

writing to /home/brewster/Desktop/cretin.v2_19_test/bowie/serial4/serial4.gen
running cretin with serial4
plotting serial4 to /home/brewster/Desktop/cretin.v2_19_test/bowie/serial4

In [7]:
# here we plot mutiple simulations into one plot to compare them


# first we create our generic object
def sim(var):

    gen2 = generator_object.User_input()

    rho, T_ev, Tkelvin = 6.9, 30*var, 512,  
    ne, z = 10**20, 12
    N0, N1 = 1, 41 #We are operating in 1d wih 40 nodes
    tin_n = 50
    E1, E2 = 4, 4.7 # the laser energies
    DN, DR = 10, .1


    gen2.materials_atom(index = 1, quantum_n_max = 3, element= "Sn")
    gen2.materials_region(nodes = [N0, N1], elec_temp = T_ev)
    gen2.materials_region_rho(rho)
    gen2.materials_region_background(ion_density = ne, electron_density = ne, average_charge = z, average_charge_squared=z**2, avg_atomic_number = tin_n )
    gen2.materials_region_element(index = 1, initial_ion_population = ne )

    gen2.geometry(type = 'slab')
    gen2.geometry_nodes(coordinate= "r", scaling_type  = "lin", nodes = [N0, N1], nodes_range=[0,DR])

    gen2.radiation_ebins(n_boundaries = 121, start = 10**-5, end = 10**(5))
    gen2.radiation_angles(n_rays = 3)

    gen2.controls(t_start=0., t_end=3*10**-6, restart = True)
    gen2.popular_switches(timestep_between_snapshot = 5)
    return gen2


i0, iend = 0,6
trials = []
name = 'trial'
# Here i showcase the mutiplot functionality automatically putting all the data into one plot per type
for i in range(i0,iend):

    trial = name+str(i)
    trials.append(trial)
    variable = 2**i
    gen2 = sim(variable)
    write_run_plot.write(name = name+str(i), object = gen2)
    write_run_plot.run(name = name+str(i), longprint=False)

serial_sim_tools.plot_all('trials_mutiplot', trials)



writing to /home/brewster/Desktop/cretin.v2_19_test/bowie/trial0/trial0.gen
running cretin with trial0

writing to /home/brewster/Desktop/cretin.v2_19_test/bowie/trial1/trial1.gen
running cretin with trial1

writing to /home/brewster/Desktop/cretin.v2_19_test/bowie/trial2/trial2.gen
running cretin with trial2

writing to /home/brewster/Desktop/cretin.v2_19_test/bowie/trial3/trial3.gen
running cretin with trial3

writing to /home/brewster/Desktop/cretin.v2_19_test/bowie/trial4/trial4.gen
running cretin with trial4

writing to /home/brewster/Desktop/cretin.v2_19_test/bowie/trial5/trial5.gen
running cretin with trial5
multiplot to /home/brewster/Desktop/cretin.v2_19_test/bowie/trials_mutiplot


In [8]:
def sim(var1, var2, var3):

    gen2 = generator_object.User_input()

    rho, T_ev, Tkelvin = 6.9*var1, 30*var2, 512*var3,  
    ne, z = 10**20, 12
    N0, N1 = 1, 41 #We are operating in 1d wih 40 nodes
    tin_n = 50
    E1, E2 = 4, 4.7 # the laser energies
    DN, DR = 10, .1


    gen2.materials_atom(index = 1, quantum_n_max = 3, element= "Sn")
    gen2.materials_region(nodes = [N0, N1], elec_temp = T_ev)
    gen2.materials_region_rho(rho)
    gen2.materials_region_background(ion_density = ne, electron_density = ne, average_charge = z, average_charge_squared=z**2, avg_atomic_number = tin_n )
    gen2.materials_region_element(index = 1, initial_ion_population = ne )

    gen2.geometry(type = 'slab')
    gen2.geometry_nodes(coordinate= "r", scaling_type  = "lin", nodes = [N0, N1], nodes_range=[0,DR])

    gen2.radiation_ebins(n_boundaries = 121, start = 10**-5, end = 10**(5))
    gen2.radiation_angles(n_rays = 3)

    gen2.controls(t_start=0., t_end=3*10**-6, restart = True)
    gen2.popular_switches(timestep_between_snapshot = 5)
    return gen2

# Here i showcase the combinatorcs functionality allowin the combination of any set of paramters for mutiplot
vars1, vars2, vars3 = np.arange(1,3,1), np.arange(4,6,1), np.arange(2,12,5)
arr = np.array([vars1, vars2, vars3], dtype=object)
combinations = serial_sim_tools.combinatorics(arr = arr)
names = []
base_name = 'combinatorics_sim'

for combination in combinations:
    [var1,var2,var3] = combination
    gen2 = sim(var1, var2, var3)
    name = f'{base_name}_{var1}_{var2}_{var3}'
    names.append(name)
    write_run_plot.write(name = name, object = gen2)
    write_run_plot.run(name = name, longprint=False)

serial_sim_tools.plot_all(f'{base_name}_multiplot', names)


writing to /home/brewster/Desktop/cretin.v2_19_test/bowie/combinatorics_sim_1_4_2/combinatorics_sim_1_4_2.gen
running cretin with combinatorics_sim_1_4_2

writing to /home/brewster/Desktop/cretin.v2_19_test/bowie/combinatorics_sim_1_4_7/combinatorics_sim_1_4_7.gen
running cretin with combinatorics_sim_1_4_7

writing to /home/brewster/Desktop/cretin.v2_19_test/bowie/combinatorics_sim_1_5_2/combinatorics_sim_1_5_2.gen
running cretin with combinatorics_sim_1_5_2

writing to /home/brewster/Desktop/cretin.v2_19_test/bowie/combinatorics_sim_1_5_7/combinatorics_sim_1_5_7.gen
running cretin with combinatorics_sim_1_5_7

writing to /home/brewster/Desktop/cretin.v2_19_test/bowie/combinatorics_sim_2_4_2/combinatorics_sim_2_4_2.gen
running cretin with combinatorics_sim_2_4_2

writing to /home/brewster/Desktop/cretin.v2_19_test/bowie/combinatorics_sim_2_4_7/combinatorics_sim_2_4_7.gen
running cretin with combinatorics_sim_2_4_7

writing to /home/brewster/Desktop/cretin.v2_19_test/bowie/combinatori